In [7]:
from seldon_deploy_sdk import Configuration, ApiClient, PredictApi, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
import pprint
import json
import requests
from requests.structures import CaseInsensitiveDict

In [8]:
SD_IP = "35.196.51.203"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'
config.verify_ssl = False
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
seldon_api_client = ApiClient(configuration=config, authenticator=auth)

In [9]:
print(config.id_token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4eEk5VXp0Slc0MFFVSklfSUhoX3NmbkRKd1VLUXc4LXRaalN0X2xqOWRrIn0.eyJleHAiOjE2NjAzNjMwNjAsImlhdCI6MTY2MDMyNzA2MCwiYXV0aF90aW1lIjowLCJqdGkiOiJhOWRiZTRhNS1kODc0LTQ5ZjItOGU4Ni00MGEwYmNiMDIyMGEiLCJpc3MiOiJodHRwOi8vMzUuMTk2LjUxLjIwMy9hdXRoL3JlYWxtcy9kZXBsb3ktcmVhbG0iLCJhdWQiOlsic2QtYXBpIiwiZGVwbG95LXNlcnZlciJdLCJzdWIiOiIxYzM5OGExYS0wNWU1LTRmMDMtYWFmZS02YTRmYzQ0Y2EwZGEiLCJ0eXAiOiJJRCIsImF6cCI6InNkLWFwaSIsInNlc3Npb25fc3RhdGUiOiJlMTAwZmUxNy0zNGY1LTRiOTktYjU3Yi0yOGM1MDkyZjdiZTIiLCJhY3IiOiIxIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJjbGllbnRJZCI6InNkLWFwaSIsImNsaWVudEhvc3QiOiIxMC4xNDIuMC4xMDYiLCJncm91cHMiOltdLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJzZXJ2aWNlLWFjY291bnQtc2QtYXBpIiwiY2xpZW50QWRkcmVzcyI6IjEwLjE0Mi4wLjEwNiJ9.BA4uns_eed-RHmmWhKgQnrQS3YoCYVFesUK6k5XM5_d_9JildZ_NDM8AXBzNKli14azwRq_hYozFaqPiQeb72_AeS3TtBEdDUHAVEFkNGickTZQkvhIMwgLxShPpRsNcNfb-yjsVl7hzbQ0Z4xl1I6pQQZbtbyOsaRyY-QANZUCAj6FjdYsr2rZao-vylinbvefGX82Vr3fuCusqg2wMwtuK503pRd-weESD6JIrIJJfhjLzmk0FxefkfW0

In [10]:
MODEL_NAME = "iris-classification"
DEPLOYMENT_NAME = f"{MODEL_NAME}"
NAMESPACE = "seldon-demos"

### Make Prediction - SDK

In [11]:
# create an instance of the API class
predict_api = PredictApi(seldon_api_client)

prediction = {
	"data": {
		"names": ["Sepal length", "Sepal width", "Petal length", "Petal Width"],
		"ndarray": [
			[6.8, 2.8, 4.8, 1.4]
		]
	}
}

try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, prediction)
    print(api_response)
except ApiException as e:
    print("Exception when calling PredictApi->predict_seldon_deployment: %s\n" % e)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.008074020139119268, 0.7781601484223357, 0.21376583143854502]]}, 'meta': {'requestPath': {'iris-classification-container': 'seldonio/sklearnserver:1.13.1'}}}


## Make a prediction using the API

In [14]:
URL = f"http://{SD_IP}/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions"
print(URL)

r = requests.post(URL, json=prediction, verify=False)

# request_id = r.headers["seldon-puid"]

# print(f"Prediction request id: {request_id}")
print(r.text)

http://35.196.51.203/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions



## Bash Example

In [15]:
%%bash

CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -s -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v1.0/predictions -d '{"data":{"names":["Sepal length","Sepal width","Petal length","Petal Width"],"ndarray":[[6.8,2.8,4.8,1.4]]}}'  | jq

In [11]:
CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v0.1/feedback -d '{"response":{"data":{"ndarray":[1]}},"truth":{"data":{"ndarray":[1]}}}'


zsh:1: command not found: kubectl
curl: (6) Could not resolve host: seldon


# Send Feedback of Request

In [17]:

# import requests module
import requests
from requests.structures import CaseInsensitiveDict

URL = f"http://{SD_IP}/seldon/seldon-demos/iris-accuracy/api/v1.0/feedback"
headers = CaseInsensitiveDict()
headers["seldon-puid"] = request_id

body = {
   "response":{
      "data":{
         "ndarray":[
            1
         ]
      }
   },
   "truth":{
      "data":{
         "ndarray":[
            0
         ]
      }
   }
}


r = requests.post(URL, headers=headers, json=body)
  
# print request object
print(r.text)

In [47]:
%%bash

CLUSTER_IP=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
curl -k -s -H "Content-Type: application/json" http://$CLUSTER_IP/seldon/seldon-demos/iris-accuracy/api/v1.0/feedback \
    -d '{"response":{"data":{"ndarray":[1]}},"truth":{"data":{"ndarray":[1]}}}'

{"data":{"tensor":{"shape":[0]}},"meta":{"requestPath":{"iris-accuracy-container":"seldonio/sklearnserver:1.13.1"}}}
